In [0]:
from pyspark.sql.functions import col, to_date, current_timestamp, lit
from pyspark.sql.types import DecimalType

BRONZE_PATH = "s3://finance-beop-fm/bronze"
SILVER_PATH = "s3://finance-beop-fm/silver/"

tables = ["orders_raw", "payments_raw", "expenses_raw","budgets_raw"]

def process_and_save_silver (table_name, bronze_df, silver_base_path):
    print(f"Processing Silver : {table_name}...")

    df = bronze_df
    for c in df.columns:
        df = df.withColumnRenamed(c, c.lower().replace(" ", "_").replace("-", "_"))
    if table_name == "orders_raw":
        df = (df.filter(col("order_amount") > 0)
              .withColumn("order_date", to_date(col("order_date")))
              .dropDuplicates(["order_id"]))
    elif table_name == "payments_raw":
        df = (df.filter(col("paid_amount") > 0)
              .filter(col("payment_status").isin("SUCCESS", "FAILED", "REFUNDED"))
              .withColumn("payment_date", to_date(col("payment_date"))))
    elif table_name == "expenses_raw":
        df = (df.filter(col("expense_amount") > 0)
              .withColumn("expense_date", to_date(col("expense_date"))))
    elif table_name == "budgets_raw":
        df = df.filter(col("budget_amount") > 0)
              
    df = df.withColumn("Silver_load_ts", current_timestamp())
    table_clean = table_name.replace("_raw", "")
    output_path = f"{silver_base_path}{table_clean}"
    df.write.mode("overwrite").parquet(output_path)

    print(f" Silver Saved {output_path}")
    print (f"Rows : {df.count():,}")

for table in tables :
    print(f"\nReading Bronze Table : {table}")
    bronze_df = spark.read.parquet(f"{BRONZE_PATH}/{table}")
    process_and_save_silver(table, bronze_df, SILVER_PATH)
    print("\nSILVER LAYER COMPLETED SUCCESSFULLY")
BRONZE_BASE_PATH = "s3://finance-beop-fm/bronze"



Reading Bronze Table : orders_raw
Processing Silver : orders_raw...
 Silver Saved s3://finance-beop-fm/silver/orders
Rows : 4,850

SILVER LAYER COMPLETED SUCCESSFULLY

Reading Bronze Table : payments_raw
Processing Silver : payments_raw...
 Silver Saved s3://finance-beop-fm/silver/payments
Rows : 7,521

SILVER LAYER COMPLETED SUCCESSFULLY

Reading Bronze Table : expenses_raw
Processing Silver : expenses_raw...
 Silver Saved s3://finance-beop-fm/silver/expenses
Rows : 118

SILVER LAYER COMPLETED SUCCESSFULLY

Reading Bronze Table : budgets_raw
Processing Silver : budgets_raw...
 Silver Saved s3://finance-beop-fm/silver/budgets
Rows : 119

SILVER LAYER COMPLETED SUCCESSFULLY
